# Notebook 4.3 - Actors curation: duplicates

This notebook gather several checks that can be run together or independently of each other. The set of these checks helps moderators to curate Duplicated actors in the SSH Open Marketplace. 

This notebook is composed of 6 sections:

0. Requirements to run this notebook
1. Get actors 
2. Duplicated actors

    2.1 Find duplicates for actors with same name and same website 
    
    2.2 Compare duplicated actors with same name and same website 
    
    2.3 Merge duplicated actors with same name and same website 
    
    2.4 Reload the Actors
    
    2.5 Find duplicates for actors with same name 
    
    2.6 Merge duplicated actors with same name 


## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [1]:
import pandas as pd #to manage dataframes
#import matplotlib.pyplot as plt #to create histograms and images
#import seaborn as sns #to create histograms and images
import numpy as np #to manage json objects
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

In [2]:
mpdata = mpd()
utils=hel.Util()
check=eva.URLCheck()

### 0.2 Utility functions

In [3]:
def getDuplicateActorsWithEmptyItems(actors):
    df_actors_ei=pd.DataFrame()
    for item in actors.itertuples():
        allEmpty=True;
        for actorid in item.idstobemerged:
            actitems=mpdata.getItemsforActor(str(actorid))
            allEmpty=allEmpty & actitems.empty
            if(not allEmpty):
                break
        if (allEmpty):
            entry = actors.loc[actors['name'] == item.name]
            df_actors_ei=pd.concat([df_actors_ei, entry])
    return df_actors_ei

In [4]:
def getDuplicateActorsWithDifferentItems(actors):
    df_results=pd.DataFrame()
    for item in actors.itertuples():  
        if (len (item.idstobemerged)<2):
            continue
        
        tempdf_sn=mpdata.getItemsforActor(str(item.idstobemerged[0])).drop_duplicates('persistentId', keep='first')
        for actorid in item.idstobemerged[1:]:
            #print (actorid)
            actitems=mpdata.getItemsforActor(str(actorid)).drop_duplicates('persistentId', keep='first')
            entry = actors.loc[actors['name'] == item.name]
            if ((tempdf_sn.empty & (not actitems.empty)) | ((not tempdf_sn.empty) & actitems.empty)):
                df_results=pd.concat([df_results, entry])
                break 
            if ((not actitems.empty)):
                tre=(actitems['persistentId'].isin(tempdf_sn['persistentId'])).value_counts()
                tre_r= (tempdf_sn['persistentId'].isin(actitems['persistentId'])).value_counts()  
                
                if ((False in tre.to_dict()) & (False in tre_r.to_dict())):
                    #print (f'{item.idstobemerged[0]}, {actorid}')
                   
                    df_results=pd.concat([df_results, entry])
                    break
    return df_results


In [5]:
import functools as ft
def getDuplicateActorsWithSameItems(dfs):
    df_actors_si=pd.DataFrame()
    act_dfs=[x for x in dfs if not x.empty]
    if (len(act_dfs)<1):
        return df_actors_si
    if (len(act_dfs)==1):
        return dfs[2]
    df_temp=ft.reduce(lambda left, right: pd.merge(left, right, on=['name'], how="outer"), act_dfs)
    if (len(act_dfs)==3):
        return df_temp.loc[(df_temp[['idstobemerged_x', 'idstobemerged_y']].isnull().all(1)) & (df_temp['idstobemerged'].notnull())][['name', 'idstobemerged']]
    if (len(act_dfs)==2):
        testr=df_temp.loc[(df_temp['idstobemerged_x'].isnull()) & 
                         (df_temp['idstobemerged_y'].notnull())][['name', 'idstobemerged_y']]
        testr.rename(columns = {'idstobemerged_y':'idstobemerged'}, inplace=True)
        return testr
    

## 1. Get actors

In [6]:
df_actors_flat =mpdata.getMPItems ("actors", False)

In [7]:
df_actors_flat.tail()

,id,name,externalIds,affiliations,website
7594,1221,Zoltan Komives,[],[],NaN
7595,8062,Zong Peng,"[{'identifierService': {'code': 'DBLP', 'label...",[],NaN
7596,218,Zoomify Inc.,[{'identifierService': {'code': 'SourceActorId...,[],NaN
7597,1590,Zoppi Angela,[],[],NaN
7598,7819,Zsófia Fellegi,"[{'identifierService': {'code': 'DBLP', 'label...",[],NaN


## 2. Duplicated actors
    2.1 Get duplicates for actors using *actor.name* and *actor.website* as filters
    2.2 Compare duplicated actors (optional)
    2.4 Merge duplicated actors

### 2.1 Get duplicates for actors using *actor.name* and *actor.website* as filter

In [8]:
utils=hel.Util()
filter_attribute=['name', 'website']
df_actor_duplicates=df_actors_flat[df_actors_flat.duplicated(subset=filter_attribute, keep=False)]
dupl_actor_website=df_actor_duplicates[df_actor_duplicates['website'].notnull()].sort_values('name')

In [9]:
print (f'Using the attributes "{filter_attribute}" as filter, there are: {dupl_actor_website.shape[0]} duplicated actors')

Using the attributes "['name', 'website']" as filter, there are: 0 duplicated actors


In [10]:
actorwebsite_tomerge=dupl_actor_website.groupby(['name','website'])['id'].apply(list).reset_index(name='idstobemerged')

/var/folders/x6/4rl3xlp91ll551m6vbkh32xm0000gn/T/ipykernel_13939/1046849184.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  actorwebsite_tomerge=dupl_actor_website.groupby(['name','website'])['id'].apply(list).reset_index(name='idstobemerged')


In [11]:
actorwebsite_tomerge.count()

name             0
website          0
idstobemerged    0
dtype: int64

In [12]:
#The number of actors with more than one duplicate
actorwebsite_tomerge[actorwebsite_tomerge.idstobemerged.map(len)>2].count()

name             0
website          0
idstobemerged    0
dtype: int64

In [13]:
actorwebsite_tomerge.head()

,name,website,idstobemerged


### 2.2 Compare duplicated actors

In [14]:
#id of duplicated actors
ids=[174, 1978]
compareitems=df_actor_duplicates[df_actor_duplicates.id.isin(ids)]

In [15]:
compareitems

,id,name,externalIds,affiliations,website


In [16]:
css_equal="font-size:1.5rem; border: 2px solid silver;background-color: white; padding: 10px 20px"
css_diff="background-color: lightyellow;  font-size:1.5rem; border: 2px solid silver; padding: 10px 20px"

In [17]:
#view items
showdiff = compareitems.T.style.apply(lambda x: [css_equal if ((len(utils.lists_to_list(x.values))==1) ) else css_diff for i in x],
                    axis=1)
showdiff

id
name
externalIds
affiliations
website


### 2.3 Merge items

The code above has generated three different dataframes:

<ol>
    <li>The dataframe <i>df_actors_with_same_items</i> that contains the duplicated actors that were attached to same items</li>
    <li>The dataframe <i>df_actors_empty_items</i> that contains the duplicated actors that were *never attached to any item*</li>
    <li> The dataframe <i>df_actors_with_different_items</i> that contains the duplicated actors that were never attached to the same itemsm, <b>the actors in this dataframe should be manually merged!</b></li>
</ol>
The function <i>postMergedActors </i> uses the API entry: <i>(POST) /api/actors/{id}/merge</i> to automatically merge the actors contained in dataframes 1 or 2.

In [ ]:
for item in actorwebsite_tomerge.itertuples():
    print(item.idstobemerged[0], ", ".join(str (e) for e in item.idstobemerged[1:]))
    mpdata.postMergedActors(str(item.idstobemerged[0]), ", ".join(str (e) for e in item.idstobemerged[1:]))

## 2.4 Reload the actor from MP

In [18]:
df_actors_flat_new =mpdata.getMPItems ("actors", False)

getting data from local repository...


### 2.5 Get duplicates for actors using actor.name as filter

In [19]:
utils=hel.Util()
filter_attribute='name'
df_actor_duplicates_new=df_actors_flat_new[df_actors_flat_new.duplicated(subset=filter_attribute, keep=False)]
actor_tomerge_new=df_actor_duplicates_new.groupby(['name'])['id'].apply(list).reset_index(name='idstobemerged')

In [30]:
print (f'Using the attributes "{filter_attribute}" as filter, there are: {actor_tomerge_new.shape[0]} duplicated actors')

Using the attributes "name" as filter, there are: 323 duplicated actors


#### _Individuating actors with the exact same name that were never attached to any items._

In [21]:
df_actors_samename_empty_items= getDuplicateActorsWithEmptyItems(actor_tomerge_new)

In [22]:
df_actors_samename_empty_items.head(10)

""


#### _Different actors with the exact same name that were never attached to at least one common item_

In [23]:
df_actors_samename_different_items=getDuplicateActorsWithDifferentItems(actor_tomerge_new)

In [25]:
df_actors_samename_different_items.count()

name             322
idstobemerged    322
dtype: int64

#### _Actors with the exact same name that were attached to at least one common item._

In [26]:
dfs=[df_actors_samename_different_items, df_actors_samename_empty_items, actor_tomerge_new]
df_actors_samename_sameitems=getDuplicateActorsWithSameItems(dfs)

In [28]:
actor_tomerge_new.sort_values('name').count()

name             323
idstobemerged    323
dtype: int64

### Compare Actors

In [ ]:
ids=[234, 2177]
compareitems=df_actor_duplicates[df_actor_duplicates.id.isin(ids)]
css_equal="font-size:1.5rem; border: 2px solid silver;background-color: white; padding: 10px 20px"
css_diff="background-color: lightyellow;  font-size:1.5rem; border: 2px solid silver; padding: 10px 20px"

In [ ]:
#view items
showdiff = compareitems.T.style.apply(lambda x: [css_equal if ((len(utils.lists_to_list(x.values))==1) ) else css_diff for i in x],
                    axis=1)
showdiff

## 2.6 Merge actors

In [ ]:
for item in df_actors_samename_sameitems.itertuples():
    print(item.idstobemerged[0], ", ".join(str (e) for e in item.idstobemerged[1:]))
    mpdata.postMergedActors(str(item.idstobemerged[0]), ", ".join(str (e) for e in item.idstobemerged[1:]))